<a href="https://colab.research.google.com/github/nurfnick/Data_Viz/blob/main/05_SQL_SubQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Sub-Queries

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


I am still gonna keep playing around with liquor data

In [2]:
%%bigquery --project pic-math
SELECT *
FROM `bigquery-public-data.iowa_liquor_sales.sales`
LIMIT 5

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,INV-37362200037,2021-06-09,4582,Jiffy #926 / Spirit Lake,"2402, 17th St",Spirit Lake,51360,POINT (-95.126585 43.42282),30,DICKINSON,1092100,Imported Distilled Spirit Specialty,421.0,SAZERAC COMPANY INC,77487,Tortilla Gold DSS,12,1000,4.85,7.28,60,436.80,60.00,15.85
1,INV-21177600024,2019-08-12,2465,Sid's Beverage Shop,2727 Dodge St,Dubuque,52003,POINT (-90.705328 42.491862),31,DUBUQUE,1081100.0,Coffee Liqueurs,65,Jim Beam Brands,67557,Kamora Coffee Liqueur,12,1000,8.39,12.59,6,75.54,6.00,1.58
2,INV-25272100048,2020-02-17,2671,Hy-Vee / Jefferson,"106, W Washington St",Jefferson,50129.0,POINT (-94.375508 42.017267),37,GREENE,1081500.0,Triple Sec,434,LUXCO INC,86507,Paramount Triple Sec,12,1000,3.84,5.76,4,23.04,4.00,1.05
3,S27599300045,2015-08-31,2191,Keokuk Spirits,1013 MAIN,KEOKUK,52632,POINT (-91.387797 40.400038),56,Lee,1081700.0,DISTILLED SPIRITS SPECIALTY,434,Luxco-St Louis,75087,Juarez Gold Dss,12,1000,4.92,7.38,48,354.24,48.00,12.68
4,S21884700116,2014-10-20,2605,Hy-Vee Drugstore #5 / Cedar Rapids,2001 BLAIRS FERRY ROAD NE,CEDAR RAPIDS,52402,POINT (-91.668909 42.034799),57,Linn,1081010.0,AMERICAN AMARETTO,421,"Sazerac Co., Inc.",71886,Amaretto E Dolce,12,750,3.34,5.01,7,35.07,5.25,1.39


Let's ask what the most expensive bottle is in each category.

In [3]:
%%bigquery --project pic-math
SELECT category_name, MAX(state_bottle_retail) as max_state_bottle_retail
FROM `bigquery-public-data.iowa_liquor_sales.sales`
GROUP BY category_name

,category_name,f0_
0,Imported Distilled Spirit Specialty,342.99
1,Coffee Liqueurs,39.72
2,Triple Sec,18.50
3,DISTILLED SPIRITS SPECIALTY,112.01
4,AMERICAN AMARETTO,27.26
...,...,...
131,MISC. IMPORTED CORDIALS & LIQUEURS,658.19
132,Temporary & Specialty Packages,13.11
133,IMPORTED VODKA - CHERRY,10.58
134,American Whiskies,28.50


This was not too hard.  Let's me mean though and ask for the second most expensive bottle of "DISTILLED SPIRITS SPECIALTY".  Here is where the sub-query is going to come into play.  We don't want to include the most expensive so instead we remove those from the computation.

In [4]:
%%bigquery --project pic-math

SELECT MAX(state_bottle_retail) as max_state_bottle_retail
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE category_name = "DISTILLED SPIRITS SPECIALTY"



,max_state_bottle_retail
0,112.01


In [9]:
%%bigquery --project pic-math
SELECT MAX(state_bottle_retail) as max_state_bottle_retail
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE state_bottle_retail != (SELECT MAX(state_bottle_retail) as max_state_bottle_retail FROM `bigquery-public-data.iowa_liquor_sales.sales` WHERE category_name = "DISTILLED SPIRITS SPECIALTY")
      AND category_name = "DISTILLED SPIRITS SPECIALTY"

,max_state_bottle_retail
0,96.51


BTW, this is not nessecarily the way I would have done this...

Let's try another.  What if we want to find how many bottles in a category are over $100.

In [5]:
%%bigquery --project pic-math

SELECT category_name, COUNTIF(state_bottle_retail>100) as number_bottles_over_benjamin
FROM `bigquery-public-data.iowa_liquor_sales.sales`
GROUP BY category_name
Order BY number_bottles_over_benjamin DESC

,category_name,number_bottles_over_benjamin
0,Scotch Whiskies,4165
1,Single Malt Scotch,2769
2,Imported Brandies,2704
3,100% Agave Tequila,2484
4,SCOTCH WHISKIES,2242
...,...,...
131,American Cordials & Liqueur,0
132,Temporary & Specialty Packages,0
133,IMPORTED VODKA - CHERRY,0
134,Delisted / Special Order Items,0


I think the book I am working from pre-dates `COUNTIF`.  Let me show you a sub-query solution.

I'll make a column asking if the bottle is over $100.

In [6]:
%%bigquery --project pic-math

SELECT *, (CASE WHEN state_bottle_retail >100 THEN 1 ELSE 0 END) as over_benjamin
FROM `bigquery-public-data.iowa_liquor_sales.sales`
LIMIT 10

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons,over_benjamin
0,INV-37362200037,2021-06-09,4582,Jiffy #926 / Spirit Lake,"2402, 17th St",Spirit Lake,51360,POINT (-95.126585 43.42282),30,DICKINSON,1092100,Imported Distilled Spirit Specialty,421.0,SAZERAC COMPANY INC,77487,Tortilla Gold DSS,12,1000,4.85,7.28,60,436.80,60.00,15.85,0
1,INV-21177600024,2019-08-12,2465,Sid's Beverage Shop,2727 Dodge St,Dubuque,52003,POINT (-90.705328 42.491862),31,DUBUQUE,1081100.0,Coffee Liqueurs,65,Jim Beam Brands,67557,Kamora Coffee Liqueur,12,1000,8.39,12.59,6,75.54,6.00,1.58,0
2,INV-25272100048,2020-02-17,2671,Hy-Vee / Jefferson,"106, W Washington St",Jefferson,50129.0,POINT (-94.375508 42.017267),37,GREENE,1081500.0,Triple Sec,434,LUXCO INC,86507,Paramount Triple Sec,12,1000,3.84,5.76,4,23.04,4.00,1.05,0
3,S27599300045,2015-08-31,2191,Keokuk Spirits,1013 MAIN,KEOKUK,52632,POINT (-91.387797 40.400038),56,Lee,1081700.0,DISTILLED SPIRITS SPECIALTY,434,Luxco-St Louis,75087,Juarez Gold Dss,12,1000,4.92,7.38,48,354.24,48.00,12.68,0
4,S21884700116,2014-10-20,2605,Hy-Vee Drugstore #5 / Cedar Rapids,2001 BLAIRS FERRY ROAD NE,CEDAR RAPIDS,52402,POINT (-91.668909 42.034799),57,Linn,1081010.0,AMERICAN AMARETTO,421,"Sazerac Co., Inc.",71886,Amaretto E Dolce,12,750,3.34,5.01,7,35.07,5.25,1.39,0
5,S14421700098,2013-09-09,2588,Hy-Vee Food and Drug #6 / Cedar Rapi,4035 MT. VERNON ROAD S.E.,CEDAR RAPIDS,52403,POINT (-91.60978 41.976835),57,Linn,1031100.0,100 PROOF VODKA,300,Mccormick Distilling Company,36908,Mccormick Vodka Pet,6,1750,7.46,11.19,3,33.57,5.25,1.39,0
6,S20340000138,2014-07-28,2590,Hy-Vee Food Store #5 / Cedar Rapids,3235 OAKLAND ROAD NE,CEDAR RAPIDS,52402,POINT (-91.658105 42.010971),57,Linn,1701100.0,DECANTERS & SPECIALTY PACKAGES,420,"Moet Hennessy USA, Inc.",1014,Glenmorangie 10YR Discovery Pack w/3 50ml,6,900,21.99,32.99,2,65.98,1.80,0.48,0
7,S27764700008,2015-10-14,2622,Hy-Vee Food Store / Iowa City,1125 N DODGE ST,IOWA CITY,52240,POINT (-91.518868 41.676095),52,Johnson,1701100.0,DECANTERS & SPECIALTY PACKAGES,420,"Moet Hennessy USA, Inc.",2140,Hennessy VS 750ml + VSOP 50ml Chain VAP,6,800,18.99,28.49,6,170.94,4.80,1.27,0
8,S03856400071,2012-02-01,3385,Sam's Club 8162 / Cedar Rapids,2605 BLAIRS FERRY RD NE,CEDAR RAPIDS,52402,POINT (-91.67969 42.031819),57,Linn,1012300.0,IRISH WHISKIES,370,Pernod Ricard USA/Austin Nichols,15627,Jameson,12,1000,17.40,26.10,60,1566.00,60.00,15.85,0
9,INV-09755600006,2018-01-11,2593,Hy-Vee Food Store / Carroll,905 US Highway 30 West,Carroll,51401,POINT (-94.853591 42.064155),14,CARROLL,1092100.0,Imported Distilled Spirit Specialty,434,LUXCO INC,75087,Juarez Gold Dss,12,1000,5.17,7.76,60,442.80,60.00,15.85,0


The new column is at the end.  I included the `LIMIT` as to reduce computation time for displaying.  I won't use that in the subquery here.

In [10]:
%%bigquery --project pic-math

SELECT category_name, SUM(over_benjamin) as number_bottles_over_benjamin
FROM (SELECT *, (CASE WHEN state_bottle_retail >100 THEN 1 ELSE 0 END) as over_benjamin
      FROM `bigquery-public-data.iowa_liquor_sales.sales`)
GROUP BY category_name
ORDER BY number_bottles_over_benjamin DESC

,category_name,number_bottles_over_benjamin
0,Scotch Whiskies,4165
1,Single Malt Scotch,2769
2,Imported Brandies,2704
3,100% Agave Tequila,2484
4,SCOTCH WHISKIES,2242
...,...,...
131,American Cordials & Liqueur,0
132,Temporary & Specialty Packages,0
133,IMPORTED VODKA - CHERRY,0
134,American Whiskies,0


Well not really sure which is faster but you get to see the `CASE` command too for adding a column.  By the way this is sometimes called an indicator variable.  When something happens (bottle over $100) you get 1 and 0 otherwise.  This is an excellent technique that we will revisist again!

Let's add some more.  Let's get the percentage of bottles over a benjamin.

In [11]:
%%bigquery --project pic-math

SELECT category_name, COUNTIF(state_bottle_retail>100) as number_bottles_over_benjamin, COUNT(*) as number_of_bottles
FROM `bigquery-public-data.iowa_liquor_sales.sales`
GROUP BY category_name
Order BY number_bottles_over_benjamin DESC

,category_name,number_bottles_over_benjamin,number_of_bottles
0,Scotch Whiskies,4165,208608
1,Single Malt Scotch,2769,111449
2,Imported Brandies,2704,271252
3,100% Agave Tequila,2484,376151
4,SCOTCH WHISKIES,2242,199770
...,...,...,...
131,Imported Flavored Vodka,0,255055
132,American Cordials & Liqueur,0,309262
133,Imported Gins,0,12
134,Delisted / Special Order Items,0,1


Now I cannot just divide those two columns because they don't exist yet!

In [16]:
%%bigquery --project pic-math

SELECT category_name, number_bottles_over_benjamin, number_of_bottles, number_bottles_over_benjamin/number_of_bottles as percent_over_benjamin
FROM (SELECT category_name, COUNTIF(state_bottle_retail>100) as number_bottles_over_benjamin, COUNT(*) as number_of_bottles
      FROM `bigquery-public-data.iowa_liquor_sales.sales`
      GROUP BY category_name)
GROUP BY category_name, number_bottles_over_benjamin, number_of_bottles
ORDER BY percent_over_benjamin DESC

,category_name,number_bottles_over_benjamin,number_of_bottles,percent_over_benjamin
0,HIGH PROOF BEER - AMERICAN,32,32,1.000000
1,Imported Whiskies,3,10,0.300000
2,JAPANESE WHISKY,31,286,0.108392
3,Iowa Distillery Whiskies,8,124,0.064516
4,SCHNAPPS - IMPORTED,1,17,0.058824
...,...,...,...,...
131,Imported Flavored Vodka,0,255055,0.000000
132,American Cordials & Liqueur,0,309262,0.000000
133,IMPORTED VODKA - CHERRY,0,14,0.000000
134,Delisted / Special Order Items,0,1,0.000000


I got here and realized this is not really the number of bottles.